In [12]:
import os
import csv
import pandas as pd
import numpy as np

import re
import datetime
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

%matplotlib inline

from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import squarify
import networkx as nx

from sklearn import model_selection, preprocessing, metrics
plt.style.use('fivethirtyeight')

print(os.getcwd())
print(os.listdir("../"))

/Users/vishakhadudani/APR1
['Untitled Folder', '.Directory-1534269790', 'APR1', 'Music', '.condarc', 'Untitled1.ipynb', '.DS_Store', 'nltk_data', 'arxiv.json', 'settings.cfg', '.CFUserTextEncoding', 'mag_2015_processed.csv', 'midterm-2-takehome.pdf', 'arxiv.csv', 'Untitled.ipynb', "NLP-Paper - CNN'.pdf", '.local', 'Creative Cloud Files', 'Pictures', 'sampleTensorFlow', 'Test_tensor.ipynb', 'processed_data.csv', '2008.json', '.ipython', 'Desktop', 'Library', '.matplotlib', 'test.py', '.oracle_jre_usage', 'MS SBU', 'Online Return Center.pdf', 'Test-Notebook.ipynb', 'MAG_processed-20181207T192808Z-002.zip', '.bash_sessions', 'Word_cloud_nlp.png', 'untitled.txt', '.bash_profile-anaconda2.bak', 'Public', 'NLP2014Paper.pdf', 'examples', 'vector_plit.png', '.anaconda', 'BF00367750.pdf', 'APR', 'Movies', 'Applications', 'result.csv', '.Trash', '.ipynb_checkpoints', '.jupyter', '.keras', 'helloworld.py', 'Documents', 'venv', 'nyc-taxi-fare-prediction.ipynb', 'mag_2018.csv', '.bash_profile', 'da

In [13]:
df = pd.read_csv('../processed_data.csv')

In [14]:
df.shape

(1435910, 21)

In [8]:
adj_list = {}
for key,row in df.iterrows():
    curr_list = adj_list.get(row["Id"], [])
    curr_list.extend(map(int,re.findall(r'\d+', df["References"][0])))
    curr_list = list(set(curr_list))
    adj_list[row["Id"]] = curr_list
adj_list

{2014935324.0: [1760544772,
  2121772044,
  2060724751,
  1508186648,
  2102415385,
  2017713688,
  263553050,
  2079854619,
  2035743770,
  2103480351,
  2092831789,
  2113467957,
  2084630589,
  2037915714,
  2148903494,
  2072542791,
  2140720209,
  1984447059,
  2058122340,
  2096285801,
  2054921835,
  1992935537,
  2125284466,
  2012304502,
  2476433526,
  2088599176,
  2050949257,
  2022581387,
  2072923279,
  2153101972,
  2047986838,
  2072020654,
  2101977779,
  2139754676,
  2137405619,
  2087687862,
  1491016379,
  2008138427,
  2070716620,
  2114171596,
  2087980253,
  1980512989,
  2026237155,
  2094679283,
  2028260084,
  2033121022,
  1977838850,
  2057008393,
  1970565395,
  1983977749,
  2052976411,
  1991855909,
  2037012265,
  2088498991,
  2088485687,
  1998521147,
  2074757955,
  2010724164,
  1986583880,
  2143841618,
  2030126423,
  2118492507,
  2032490333,
  1988829538,
  2080870246,
  2018317672,
  2092377962,
  2148226925,
  1997613934,
  2058323824,
  20119

In [9]:
outlink_map = {}
inlink_map = {}
df_new = df[df["References"].notnull()]
for index, row in df_new.iterrows():
    ref = row["References"]
    ref_list = list(map(int,ref.strip("[]'").split(",")))
    for ref in ref_list:
        listi = inlink_map.get(ref, [])
        listi.append(row["Id"])
        inlink_map[ref] = listi

KeyboardInterrupt: 

In [ ]:
rev_adj_list = {}
for key, nodes in adj_list.items():
    for n in nodes:
        curr_list = rev_adj_list.get(n, [])
        curr_list.append(key)
        curr_list = list(set(curr_list))
        rev_adj_list[n] = curr_list

In [ ]:
def get_reach(id):
    nodes_list = inlink_map[id]
    temp_list = []
    result_list = []
    for i in range(3):
        for n in nodes_list:
            if n in inlink_map:
                temp_list.extend(inlink_map[n])
            result_list.append(n)
        nodes_list = list(set(temp_list) - set(result_list)-set([id]))
    return len(result_list)

In [ ]:
df['reach'] = df.apply(lambda row: get_reach(row["Id"]),axis = 1)   